In [7]:
from common_functions import google_sheets, get_secret, ret_metabase
from datetime import datetime, timedelta
import os
import sqlalchemy
import psycopg2
import json
from pathlib import Path
import pandas as pd

In [2]:
def initialize_env():
    """
    Initialize environment variables and credentials for Snowflake, Slack, Metabase, and Google Sheets.
    Reads secrets and sets them as environment variables for use in other functions.
    """
    snowflake_sg_secret = json.loads(get_secret("Snowflake-sagemaker"))
    slack_secret = json.loads(get_secret("prod/slack/reports"))
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    dwh_writer_secret = json.loads(get_secret("prod/db/datawarehouse/sagemaker"))

    os.environ["SNOWFLAKE_USERNAME"] = snowflake_sg_secret["username"]
    os.environ["SNOWFLAKE_PASSWORD"] = snowflake_sg_secret["password"]
    os.environ["SNOWFLAKE_ACCOUNT"] = snowflake_sg_secret["account"]
    os.environ["SNOWFLAKE_DATABASE"] = snowflake_sg_secret["database"]

    os.environ["SLACK_TOKEN"] = slack_secret["token"]

    os.environ["FINTECH_EMONEY_EMAIL"] = fintech_service_account["email_name"]
    os.environ["FINTECH_EMONEY_PASSWORD"] = fintech_service_account["email_password"]

    metabase_secret = json.loads(get_secret("prod/metabase/maxab_config"))
    os.environ["EGYPT_METABASE_USERNAME"] = metabase_secret["metabase_user"]
    os.environ["EGYPT_METABASE_PASSWORD"] = metabase_secret["metabase_password"]

    os.environ["DWH_WRITER_HOST_NEW"] = dwh_writer_secret["host"]
    os.environ["DWH_WRITER_NAME_NEW"] = dwh_writer_secret["dbname"]
    os.environ["DWH_WRITER_USER_NAME_NEW"] = dwh_writer_secret["username"]
    os.environ["DWH_WRITER_PASSWORD_NEW"] = dwh_writer_secret["password"] 

    json_path_sheets = str(Path.home()) + "/service_account_key_sheets.json"
    sheets_key = get_secret("prod/maxab-sheets")
    f = open(json_path_sheets, "w")
    f.write(sheets_key)
    f.close()
    os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"] = json_path_sheets

In [3]:
initialize_env()

In [4]:
def clean_column_id(df, column_name):
    """
    Clean a DataFrame column by removing commas and converting to integer type if possible.
    Args:
        df (pd.DataFrame): DataFrame to clean
        column_name (str): Name of the column to clean
    Returns:
        pd.DataFrame: DataFrame with cleaned column
    """
    # Ensure the column is treated as a string
    df[column_name] = df[column_name].astype(str)
    
    # Replace commas in the string
    df[column_name] = df[column_name].str.replace(',', '')
    
    # Convert back to an integer, if appropriate
    df[column_name] = df[column_name].astype('Int64', errors='ignore')
    
    return df

In [24]:
date = "June 22"
day = -1
now = datetime.now() + timedelta(days=day)
print(now)


2025-06-22 11:25:19.025049


In [14]:
agents = google_sheets(date, "Data", "get")

/home/ec2-user/service_account_key.json


In [16]:
agents = agents[["Agent", "Agent_id"]]

In [9]:

    host = os.environ["DWH_WRITER_HOST_NEW"]
    database = os.environ["DWH_WRITER_NAME_NEW"]
    user = os.environ["DWH_WRITER_USER_NAME_NEW"]
    password = os.environ["DWH_WRITER_PASSWORD_NEW"]

    conn = psycopg2.connect(host=host, database=database, user=user, password=password)
    print("Successfully connected to DB")

    now = datetime.now() + timedelta(hours = 3)
    df = ret_metabase("Egypt", 61071)
    df.columns = df.columns.str.lower()

    df['dispatched_at'] = now
 
    # db connection:
    engine = sqlalchemy.create_engine(f"postgresql+psycopg2://{user}:{password}@{host}/{database}")
    print(bool(engine))



    # update the table on db:
    with engine.connect() as conn:
        print("start")
        df.to_sql(name='task_based_am_projects', schema='fintech', con=engine, if_exists='append', chunksize=1000, method='multi', index=False)
        print("end")
        conn.close()

        # close the connection:
        conn.close()
    print(now)
    print("Agent dispatching process complete.")

Successfully connected to DB
/home/ec2-user/service_account_key.json
True
start
end
2025-06-24 17:59:51.276536
Agent dispatching process complete.
